# Problem 3 Sample Code

This sample code is meant as a guide on how to use PyTorch and how to use the relevant model layers. This not a guide on how to design a network and the network in this example is intentionally designed to have poor performace.

In [74]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
from torch.utils.data import TensorDataset, Dataset, DataLoader
import seaborn as sns
from sklearn.model_selection import train_test_split

In [75]:
df = pd.read_csv('out.csv')
df = df.drop(columns= "uid")
print("===dataframe=== \n {}".format(df))
# print ("df after drop \n{}".format(df))
# df['split'] = np.random.randn(df.shape[0], 1)
# msk = np.random.rand(len(df)) <= 0.7

# train = df[msk]
# train.drop(['split'], axis=1, inplace=True)
# test = df[~msk]
# test.drop(['split'], axis=1, inplace=True)
X = np.array(df.drop(columns=["label"]))
print(" - pre X \n {} - ".format(X))

# def normalize(x):
#     x = np.asarray(x)
#     return (x - x.min()) / (np.ptp(x))
# X_normed = []
# for i in range(0, len(X)):
#     X_normed.append(X[i])
# # X = (X - X.mean())/(X.std())
# print(" - NORMALIZED  X \n {} - ".format(X_normed))

y = np.array(df["label"])

train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.2)
print(" pre TRAIN X \n {}".format(train_X))

train_X = torch.tensor(train_X, dtype=torch.float32)
print("TENSOR  X \n {}".format(train_X))
train_y = torch.tensor(train_y, dtype=torch.float32)
# 80/20 x values as floats
test_X = torch.tensor(test_X, dtype=torch.float32)
test_y = torch.tensor(test_y, dtype=torch.float32)

train_dataset = TensorDataset(train_X, train_y)
test_dataset = TensorDataset(test_X, test_y)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=True) 

print ("x \n{}".format(df))
print ("y \n{}".format(df))

# x_train = torch.tensor(train[['chi','lin_reg','lin_reg_chi','stddev_step_speed']].copy().to_numpy(), dtype=torch.float32)
# y_train = torch.tensor(train['label'].copy().to_numpy(), dtype=torch.long)
# x_test = torch.tensor(train[['chi','lin_reg','lin_reg_chi','stddev_step_speed']].copy().to_numpy(), dtype=torch.float32)
# y_test = torch.tensor(train['label'].copy().to_numpy(), dtype=torch.long)
# train_dataset = TensorDataset(x_train, y_train)
# test_dataset = TensorDataset(x_test, y_test)

===dataframe=== 
        label       chi   lin_reg  lin_reg_chi  stddev_step_speed
0          0  0.000920  0.992533     0.000065           0.154567
1          0  0.000228  0.973844     0.000128           0.154105
2          0  0.000256  0.894242     0.000435           0.154033
3          0  0.000088  0.967026     0.000379           0.154711
4          0  0.006075  0.000296     0.000100           0.153187
...      ...       ...       ...          ...                ...
16075      0  0.005071  0.999245     0.000057           0.160588
16076      0  0.014942  0.999707     0.000089           0.232843
16077      0  0.024277  0.998892     0.000236           0.211919
16078      0  0.007277  0.998953     0.000045           0.145301
16079      0  0.006620  0.999056     0.000480           0.267385

[16080 rows x 5 columns]
 - pre X 
 [[9.20302016e-04 9.92532952e-01 6.50811419e-05 1.54567320e-01]
 [2.27743852e-04 9.73843667e-01 1.27606689e-04 1.54105100e-01]
 [2.55930344e-04 8.94242185e-01 4.34696

We can convert images to numpy arrays and plot them with matplotlib:

## Network Definition
Let's instantiate a model and take a look at the layers.

In [107]:
model = nn.Sequential(
    nn.Linear(4, 10),
    nn.Sigmoid(),
    nn.Linear(10, 5),
    nn.Sigmoid(),
    nn.Linear(5, 1),
    nn.Softmax(dim = 1)
)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
loss_fn = nn.BCELoss()
# loss_fn = nn.CrossEntropyLoss()
torch.manual_seed(15552494823729223621)

## Training
We also choose an optimizer and a loss function.

We could write our training procedure manually and directly index the `Dataset` objects, but the `DataLoader` object conveniently creates an iterable for automatically creating random minibatches:

We now write our backpropagation loop, training for 10 epochs.

In [108]:
model.train()

for epoch in range(20):
    for batch_idx, (data, target) in enumerate(train_loader):
        # print("data \n{}".format(data))
        # Erase accumulated gradients
        optimizer.zero_grad()

        # Forward pass
        output = model(data)
        # print(" -- OPUTPUT -- \n{}".format(output))
        output = output.flatten()
        print(" -- OPUTPUT -- \n{}".format(output))
        # print(" -- TARGET -- \n{}".format(target))

        # Calculate loss
        loss = loss_fn(output, target)

        # Backward pass
        loss.backward()
        
        # Weight update
        optimizer.step()

    # Track loss each epoch
    print('Train Epoch: %d  Loss: %.4f' % (epoch + 1,  loss.item()))

 -- OPUTPUT -- 
tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       grad_fn=<ReshapeAliasBackward0>)
 -- OPUTPUT -- 
tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       grad_fn=<ReshapeAliasBackward0>)
 -- OPUTPUT -- 
tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       grad_fn=<ReshapeAliasBackward0>)
 -- OPUTPUT -- 
tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       grad_fn=<ReshapeAliasBackward0>)
 -- OPUTPUT -- 
tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       grad_fn=<ReshapeAliasBackwar

RuntimeError: all elements of input should be between 0 and 1

## Testing
We can perform forward passes through the network without saving gradients.

In [ ]:
# Putting layers like Dropout into evaluation mode
model.eval()

test_loss = 0
correct = 0

# Turning off automatic differentiation
with torch.no_grad():
    for data, target in test_loader:
        output = model(data)
        test_loss += loss_fn(output, target).item()  # Sum up batch loss
        pred = output.argmax(dim=1, keepdim=True)  # Get the index of the max class score
        correct += pred.eq(target.view_as(pred)).sum().item()

test_loss /= len(test_loader.dataset)

print('Test set: Average loss: %.4f, Accuracy: %d/%d (%.4f)' %
      (test_loss, correct, len(test_loader.dataset),
       100. * correct / len(test_loader.dataset)))

Test set: Average loss: nan, Accuracy: 5733/11300 (50.7345)
